In [ ]:
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

from typing import List

import model_eld_transfer

degree_sign = u'\N{DEGREE SIGN}'

In [ ]:
ft_percents = [0.0, 0.2, 0.4, 0.6, 0.8]
shuffle_seeds = [17, 42, 64, 76, 98]

In [ ]:
def plot_multiseed_error_pir(ft_percents: List[float], from_building: str, from_tower: int, to_building: str, season: str = None):
    # MAKE PREDICTIONS AND PLOT TRENDS
    rmse_list = []
    # [tow-1] [seed index] [error]

    for tow in range(1,3):
        to_features = [f'{to_building}_Tower_{tow} enteringWaterTemp', f'{to_building}_Tower_{tow} outdoorAirDryBulb', f'{to_building}_Tower_{tow} outdoorAirWetBulb', f'{to_building}_Tower_{tow} vfdPercent', f'{to_building}_Tower_{tow} fanA_vfdPower', f'{to_building}_Tower_{tow} fanB_vfdPower', f'{to_building}_Tower_{tow} dayOfWeek', f'{to_building}_Tower_{tow} hourOfDay', f'{to_building}_Tower_{tow} efficiency']
        to_target = f'{to_building}_Tower_{tow} leavingWaterTemp'
        rmse_list.append([])

        for s, seed in enumerate(shuffle_seeds):
            rmse_list[tow-1].append([])

            for ft_percent in ft_percents:
                rmse, fig, mabs = model_eld_transfer.eld_transfer(from_building_name=from_building, from_tower_number=from_tower, to_building_name=to_building, to_tower_number=tow, to_features=to_features, to_target=to_target, to_season=season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                rmse_list[tow-1][s].append(rmse)


    # PLOT ERROR GRADIENT

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'RMSE with seed {shuffle_seeds[0]}': rmse_list[0][0], 
                                f'RMSE with seed {shuffle_seeds[1]}': rmse_list[0][1],
                                f'RMSE with seed {shuffle_seeds[2]}': rmse_list[0][2],
                                f'RMSE with seed {shuffle_seeds[3]}': rmse_list[0][3],
                                f'RMSE with seed {shuffle_seeds[4]}': rmse_list[0][4],})
    fig1 = px.line(print_df, y=[f'RMSE with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Errors')
    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'RMSE with seed {shuffle_seeds[0]}': rmse_list[0][0], 
                                f'RMSE with seed {shuffle_seeds[1]}': rmse_list[0][1],
                                f'RMSE with seed {shuffle_seeds[2]}': rmse_list[0][2],
                                f'RMSE with seed {shuffle_seeds[3]}': rmse_list[0][3],
                                f'RMSE with seed {shuffle_seeds[4]}': rmse_list[0][4],})
    fig2 = px.line(print_df, y=[f'RMSE with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Errors')

    figfinal = make_subplots(rows=1, cols=2)
    for dt in fig1.data:
        figfinal.add_trace(dt, row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Error", row=1, col=1)

    for dt in fig2.data:
        figfinal.add_trace(dt, row=1, col=2)
    figfinal.add_trace(fig2.data[1], row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Error", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (LSTM Autoencode)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/feature_extraction/eld_ft/{from_building}{from_tower}_to_{to_building}{season}_gradient.html")

    # PLOT PERFORMANCE IMPROVEMENT RATIO

    pir = [[[(seed_rmses[0] - rmse) / seed_rmses[0] for rmse in seed_rmses] for seed_rmses in tower_rmses] for tower_rmses in rmse_list]

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'PIR with seed {shuffle_seeds[0]}': pir[0][0], 
                                f'PIR with seed {shuffle_seeds[1]}': pir[0][1],
                                f'PIR with seed {shuffle_seeds[2]}': pir[0][2],
                                f'PIR with seed {shuffle_seeds[3]}': pir[0][3],
                                f'PIR with seed {shuffle_seeds[4]}': pir[0][4],})
    fig1 = px.line(print_df, y=[f'PIR with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Performance improvement ratio')

    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'PIR with seed {shuffle_seeds[0]}': pir[1][0], 
                                f'PIR with seed {shuffle_seeds[1]}': pir[1][1],
                                f'PIR with seed {shuffle_seeds[2]}': pir[1][2],
                                f'PIR with seed {shuffle_seeds[3]}': pir[1][3],
                                f'PIR with seed {shuffle_seeds[4]}': pir[1][4],})
    fig2 = px.line(print_df, y=[f'PIR with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Performance improvement ratio')

    figfinal = make_subplots(rows=1, cols=2)
    for dt in fig1.data:
        figfinal.add_trace(dt, row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=1)

    for dt in fig2.data:
        figfinal.add_trace(dt, row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (LSTM Autoencoder)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/feature_extraction/eld_ft/{from_building}{from_tower}{season}_to_{to_building}{season}_pir.html")

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="Kissam", season="summer")

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=2, to_building="Kissam", season="summer")

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="ESB", season="summer")

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=2, to_building="ESB", season="summer")

## --- Testing on MRB ---

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="MRB", season="summer")

In [ ]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="MRB", season="summer")